Video 56: Recommender System Code Along Project
=====

In [2]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('recommender system').getOrCreate()
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
data = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [3]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [5]:
training, test = data.randomSplit([0.8, 0.2])
als = ALS(maxIter=5, regParam=0.1, userCol='userId', itemCol='movieId', ratingCol='rating')
model = als.fit(training)
predictions = model.transform(test)
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|    26| 0.4940685|
|     31|   1.0|    27| 1.3831553|
|     31|   1.0|    19| 1.2023821|
|     31|   1.0|     4| 1.2745703|
|     31|   1.0|    18|0.05314377|
|     85|   1.0|    26| 2.9504766|
|     85|   1.0|    15|0.97421277|
|     65|   1.0|    16| 0.6760448|
|     65|   2.0|     3| 0.7931684|
|     65|   5.0|    23| 1.3888613|
|     53|   1.0|    23|0.35662287|
|     34|   1.0|    15| 1.0865842|
|     34|   3.0|    25| 0.8939153|
|     81|   1.0|    22| 1.7356654|
|     81|   1.0|     1| 1.4753563|
|     81|   2.0|     9| 1.8591249|
|     81|   1.0|    21|  1.669753|
|     28|   1.0|     5|  2.135886|
|     28|   2.0|    15| 1.9883984|
|     28|   1.0|    17| 1.6204515|
+-------+------+------+----------+
only showing top 20 rows



In [7]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
rmse = evaluator.evaluate(predictions)
print('RMSE:')
print(rmse)

RMSE:
1.0300876437813553


In [9]:
single_user = test.filter(test['userId']==11).select(['movieId', 'userId'])
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     19|    11|
|     25|    11|
|     43|    11|
|     45|    11|
|     59|    11|
|     70|    11|
|     71|    11|
|     76|    11|
|     79|    11|
|     86|    11|
+-------+------+



In [11]:
recommendations = model.transform(single_user)
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     79|    11|  2.928665|
|     76|    11|  2.633119|
|     19|    11|  2.593341|
|     43|    11| 1.8047391|
|     70|    11| 1.6373996|
|     71|    11| 1.5395231|
|     45|    11| 1.4104843|
|     86|    11|  1.256038|
|     25|    11|0.78380525|
|     59|    11| 0.7598884|
+-------+------+----------+

